# Finding Mentions by Name

In this lab we will try to identify frequencies of proper nouns in the news corpus. More specifically we are looking for names that occur frequently in the news. This is an approximate analysis and you will get slightly different results by adjusting the parameters.

You will learn how to:
- identify words based on capitalization
- work with regular expressions
- compute Levenshtein distance

## Prerequisites

This lab requires following 3rd party libraries! Run this command before starting:

```
pip install python-Levenshtein
```

In [1]:
# import dependencies 
import re
from corpus_utilities import load_articles
from Levenshtein import distance as levenshtein_distance

## 1. Read article data from corpus

In [2]:
# path to corpus directory
# change this value as necessary
directory_path = '../corpus'

# use utility script to load articles
articles = load_articles(directory_path)

print('Sanity check! Got', len(articles), 'articles.')  

Sanity check! Got 67259 articles.


## 2. Naive search approach using Regular Expression

Let's use capitalization rules as a way to identify nouns. When a word occurs in the beginning of the sentence, it is impossible to tell whether capitalization is an indication of a word being a proper noun, or just the first word of a sentence, so we will accept those words as well. Therefore this is a very naive approach to finding proper nouns, but it will get us started. 

In [3]:
# Regular expression pattern for matching capitalized words
# Initially let's match all capitalized word sequences 
pattern = r'([A-ZÄÖ][a-zäö]+(?=\s[A-ZÄÖ])(?:\s[A-ZÄÖ][a-zäö]+)+)'

matches = []

for article in articles:

    [title, summary] = article[1:3]
    
    # find capitalized words in article title
    title_words = re.findall(pattern, title)
  
    # find capitalized words in article summary
    summary_words = re.findall(pattern, summary)
    
    # combine all matches
    proper_nouns = title_words + summary_words
    
    if len(proper_nouns) > 0:
        matches += proper_nouns


# show stats about findings        
print('Found', len(matches), 'capitalized word seaquences.')
print('Found', len(list(set(matches))), 'unique capitalized word seaquences.', '\n')

# display some matches on the screen to observe the results
words_to_show, cols = 80, 2
preview = list(set(matches))[0:words_to_show]
display = lambda x, y: str(matches.count(x)).ljust(5, ' ') + ' ' + x.ljust(y,' ')

for i in range(0, int(words_to_show / cols)):
    upper, lower = i * cols, (i + 1) * cols
    row_words = [display(p, 45) for p in preview[upper:lower]]
    print(' '.join(row_words))

Found 73575 capitalized word seaquences.
Found 31362 unique capitalized word seaquences. 

1     Enter Airin                                   1     Patricia Routledge                           
8     Julius Honka                                  2     Sharon Tate                                  
2     Atletico Madridin                             4     Antonio Giovinazzin                          
1     Ina Forsmanin                                 1     Jos Finnpulpin                               
1     Chris Froomelle                               1     Metallican Suomen                            
1     Valtteri Bottaksen Silverstonessa             2     Adonis Stevenson                             
2     Sota Ukrainassa                               3     Jeff Bezosin                                 
2     Pete Kauppinen                                2     Jennifer Arcuri                              
1     Muistatko Kotikadun Susanna                   2     Dorothea Wiereristä

These results show this approach does identify proper nouns but this approach has specific issues:

- the discovered words include extra words: `Juontaja`
- we cannot automatically assume a fixed length word sequence: `Valerie Morris Campbell` vs. `Frederik`
- the different word forms prevent use from grouping matches properly: `Petteri Orpolta` vs. `Petteri Orpo`

## 3. Try frequency based sorting

Observing the results above, we can see that actual names of people seem to have higher frequency than random word pairs. Based on this observation, lets sort the previously discovered proper noun words by frequency and see what happens.

In [4]:
words_to_show, cols = 100, 3
sorted_matches = sorted([(matches.count(w), w) for w in list(set(matches))], reverse=True)

for i in range(0, int(words_to_show / cols)):
    upper, lower = i * cols, (i + 1) * cols
    row_words = [display(p[1], 30) for p in sorted_matches[upper:lower]]
    print(' '.join(row_words))
    
print('\nWords with 2 or more occurrences:', len([p for p in sorted_matches if p[0] > 1]))    

537   Kimi Räikkönen                 406   Valtteri Bottas                388   Sensuroimaton Päivärinta      
344   Kimi Räikkösen                 259   Patrik Laine                   210   Teemu Pukki                   
206   Valtteri Bottaksen             201   Kaisa Mäkäräinen               196   Matti Nykäsen                 
195   Antti Rinteen                  188   Iivo Niskanen                  183   Lewis Hamilton                
181   Antti Rinne                    180   Kaapo Kakko                    166   Matti Latvala                 
162   Patrik Laineen                 159   Krista Pärmäkoski              151   Donald Trump                  
151   Big Brother                    137   Teemu Pukin                    136   Therese Johaug                
132   Jussi Halla                    128   Prinssi Harry                  124   New Yorkissa                  
123   Herttuatar Meghan              119   Olli Lindholmin                117   Sauli Niinistö                
1

This result is already much better. We have narrowed down from > 70K capitalized words to < 8K most frequent expressions. 

But there are still some issues we should address. The following conjugations are expressions referring to the same person.

- `Kimi Räikkönen` and `Kimi Räikkösen`
- `Valtteri Bottas` and `Valtteri Bottaksen`
- `Matti Nykänen` and `Matti Nykäsen`
- `Antti Rinteen` and `Antti Rinne`

The frequency count would be more accurate, if we were able identify relationship between these words.

## 4. Grouping similar words

[Levenshtein distance](https://en.wikipedia.org/wiki/Levenshtein_distance) is used for calculating the distance between words. Let's try to improve our result by grouping words based on this calculated word distance. 

Next let's try to cluster related words, to identify conjugations, by using the Levenshtein distance. We will only allow a very small Levenshtein distance (`max_dist`), which gives us high confidence that two words are related. We will perform number of passes over the most frequently occurring words, until we are no longer able to find new relationships. **Caution: This operation will take a while to finish.**

In [5]:
# distionary to hold related terms
related_terms = {}

# copy the list of unique sorted capitalized words; drop frequency count
search_base = [w[1] for w in sorted_matches[:]]

# maximum allowed Levenshtein distance
# increasing this number will create more relationships
# decreasing this number will make the realtionships more strict
max_dist = 2

# count number of passes
iter_count = 0

# Group top N most frequent words 
top_words = 1000

# when an iteration results in fewer word pair discoveries 
# than indicated by this number, stop iterating
stop_diff = 5


while True:

    additions = 0
    iter_count += 1
    print('Grouping related words....', iter_count)
    
    # search top N words that are yet to be paired
    for w1 in search_base[0:top_words]:

        dict_match = False

        # first look for closely related terms in the dictionary
        for k, words in related_terms.items():
            for w2 in words:
                
                dist = levenshtein_distance(w1, w2)

                if dist > 0 and dist <= max_dist:
                    if w1 not in related_terms[k]:
                        related_terms[k].append(w1)
                    if w1 in search_base:
                        search_base.remove(w1)
                    dict_match = True
                    additions += 1

        # if good match was in dictionary stop here
        if dict_match:
            continue

        closest_matches = [w1]

        # look for best match in sorted words list
        # these are words that are yet to be paired
        for w2 in search_base:

            # ignore exact match
            if w2 == w1:
                continue

            dist = levenshtein_distance(w1, w2)

            # pair all close matches
            if dist <= max_dist:
                closest_matches.append(w2)

        # when we find at least 1 close match
        # move these closely related words to dictionary
        if len(closest_matches) > 1:
            key = len(related_terms.keys()) + 1
            related_terms[key] = list(set(closest_matches))
            for w in closest_matches:
                if w in search_base:
                    search_base.remove(w)
            additions += 1            
    
    if additions < stop_diff:
        break
        
# display results
print('\nFound', len(related_terms.keys()), 'related words!', '\n')              

Grouping related words.... 1
Grouping related words.... 2
Grouping related words.... 3
Grouping related words.... 4
Grouping related words.... 5
Grouping related words.... 6
Grouping related words.... 7
Grouping related words.... 8
Grouping related words.... 9
Grouping related words.... 10

Found 1803 related words! 



In [6]:
# display dictionary contents
for i in range(0, 50):
    v = list(related_terms.values())[i]
    output = ', '.join(v)
    dots = ('...' if len(output) > 100 else '')
    print((str(i + 1) + '.').ljust(3, ' '), output[0: 100].strip() + dots) 

1.  Kimin Räikkösen, Kimi Räikköseen, Kimi Räikkönen, Kimi Räikkösen, Kimi Räikköstä
2.  Valtteri Bottasta, Valtteri Bottas
3.  Patrik Laine, Patrik Laineen
4.  Teemu Pukin, Teemu Pukki, Teemu Pukkia
5.  Valtterin Bottaksen, Valtteri Bottakseen, Valtteri Bottaksen
6.  Kaisa Mäkäräisen, Kaisa Mäkäräinen, Kaisa Mäkäräiseen, Kaisa Mäkäräistä, Kaisa Mäkäräisestä
7.  Matti Nykäseen, Matti Nykänen, Matti Nykäsen, Matti Nykästä, Matti Nykäsestä
8.  Antti Rintanen, Antti Rinteen
9.  Ivo Niskanen, Iio Niskanen, Iivo Niskanen, Iivo Niskasen, Iivo Niskasta
10. Lewis Hamilton, Lewis Hamiltonin, Lewis Hamiltonia, Lewis Hamiltonista, Lewis Hamiltoniin
11. Kaapo Kakkoon, Kaapo Kakkoa, Kaapo Kakon, Kaapo Kakko
12. Matti Latvalaa, Matti Latvala, Matti Latvalle, Matti Latvan, Maari Latvala, Matti Latvalan, Matti La...
13. Krista Pärmäkoski, Krista Pärmäkosken, Krista Pärmäkoskea, Krista Pärmäkoskelta, Krista Pärmäkoskest...
14. Donald Trumpia, Donald Trumpin, Donald Trump
15. Big Brotherin, Big Brother,

Observing these results, we can see results look generally good and we can see different word conjugations grouped together. We are also able to identify relationships when when there are minor spelling mistakes (`Iivo Niskanen` `Ivo Niskanen`,`Iio Niskanen`). 

The previous step removes a lot of noise and allows us to narrow down to dictionary of related proper nouns. There is some overlap in this dictionary, so let's perform word distance calculation around the values within the dictionary only, to establish a few more relationships and merge duplicate values.

In [7]:
# if we find existing values in dictionary
# where word distance between any pairing of words
# is less of equal to this boundary, we will group
# those terms within the dictionary. 
term_dist = 2
merge_counter = 0
merged_dict = {}
check_keys = list(related_terms.keys())

while len(check_keys) > 0:
    
    k1 = check_keys.pop()    
    values1 = related_terms[k1]
    source_dict, closest_match, min_dist = None, None, term_dist + 1
    
    for w1 in values1:   
        for k2 in merged_dict.keys():
            values2 = merged_dict[k2]
            for w2 in values2:
                # when working with a limited set of words
                # if one contains the other assume they are related
                if w1 in w2 or w2 in w1:
                    dist = 0
                else:
                    dist = levenshtein_distance(w1, w2)
                if dist < min_dist:
                    min_dist = dist
                    closest_match = k2  
                    source_dict = merged_dict
        
        if min_dist > 0:
            for k2 in check_keys:
                values2 = related_terms[k2]
                for w2 in values2:
                    if w1 in w2 or w2 in w1:
                        dist = 0
                    else:
                        dist = levenshtein_distance(w1, w2)
                    if dist < min_dist:
                        min_dist = dist
                        closest_match = k2
                        source_dict = related_terms

    if min_dist <= term_dist:
        if merge_counter < 20:
            print('merging', min_dist, \
                  (', '.join(values1))[0:40].ljust(40,' '), '<-', \
                  (', '.join(source_dict[closest_match]))[0:40])
            
        merged_dict[k1] = values1 + source_dict[closest_match]
        if closest_match in check_keys:
            check_keys.remove(closest_match)
        merge_counter += 1
    else:
        merged_dict[k1] = values1
        
    # ensure no duplicates
    merged_dict[k1] = list(set(merged_dict[k1]))
    
        
print('\nMerged', merge_counter, 'values')    

merging 0 Johannes Kläbolle, Johannes Kläbolla     <- Johannes Kläbota, Johannes Kläbon, Johan
merging 0 Joonas Donskoilla, Joonas Donskoille     <- Joonas Donskoi, Joonas Donskoin, Joonas 
merging 0 Juontaja Roope Salminen, Juontajina Roop <- Roope Salminen, Roope Salmisen, Roope Sa
merging 2 Jutta Urpilaista, Jutta Urpilaisesta     <- Jutta Urpilaiseen, Jutta Urpilaisen, Jut
merging 0 Jääkiekkoilija Mikael Granlund, Jääkiekk <- Mikael Granlundin, Mikael Granlundia, Mi
merging 0 Kansanedustaja Krista Kiuru, Kansanedust <- Krista Kiurun, Krista Kiuru, Krista Kiur
merging 0 Kansanedustaja Sebastian Tynkkysen, Kans <- Sebastian Tynkkynen, Sebastian Tynkkysen
merging 0 Kanye Westillä, Kanye Westiltä           <- Kanye West, Kanye Westin
merging 0 Kiekkolegenda Raimo Helmisen, Kiekkolege <- Raimo Helminen, Raimo Helmisen, Raipe He
merging 0 Kimi Räikkösen Alfa Romeo, Kimi Räikköse <- Kimin Räikkösen, Kimi Räikköseen, Kimi R
merging 0 Kohujuoksija Caster Semenyan, Kohujuoksi <- Caster Semen

There are still some limitations, for example: 
- `Sauli Niinistö` and `Presidentti Niinistö` being grouped separately; and 
- `Miss Suomi` and `Missä Suomi` being grouped together

Given the complexity of this exercise and accepting that this method produces a high approximation - not exact matches - we will move on with these results. It is certainly faster than trying to do this manually. Let's figure out who are the most discussed people in the Finnish media!

## 5. Identify most discussed people

In this last step we will use `merged_dict` to compute the total frequencies, this will allow us to discover the "most discussed people" over time. Since we are performing this search over the entire corpus, this will give us a count over all time.

In [8]:
# create a list to hold final results
result = []

for words in merged_dict.values():
    
    top_word_form, highest_frequency = None, 0
    article_matches = []

    for word in words:
        word_freq = 0
        
        for i in range(0, len(articles)):
            [title, summary] = articles[i][1:3]
            if word in title or word in summary:
                word_freq += 1
                if i not in article_matches:
                    article_matches.append(i)

        # accept the most frequent word form as default spelling 
        if top_word_form == None or word_freq > highest_frequency:
            highest_frequency = word_freq
            top_word_form = word
    
    total_frequency = len(article_matches)
    result.append((total_frequency, top_word_form))

# sort by popularity with top expression first    
result = sorted(result, reverse=True)    
    
# Display results
print('=' * 50)
print('All-Time Mentions, Top 100'.upper())
print('=' * 50, '\n')
print('#'.ljust(5), 'articles'.ljust(10), 'Topic')

for i in range(0, 100):
    
    (count, word) = result[i]  
    
    print((str(i + 1 ) + '.').ljust(10), str(count).ljust(5), word)

ALL-TIME MENTIONS, TOP 100

#     articles   Topic
1.         727   Kimi Räikkönen
2.         701   Kimi Räikkönen
3.         682   Kimi Räikkönen
4.         682   Kimi Räikkönen
5.         365   New York
6.         365   New York
7.         365   New York
8.         365   New York
9.         365   New York
10.        365   New York
11.        364   Patrik Laine
12.        364   Patrik Laine
13.        322   Valtteri Bottas
14.        308   Donald Trump
15.        308   Donald Trump
16.        308   Donald Trump
17.        286   Teemu Pukki
18.        279   Kaisa Mäkäräinen
19.        275   Iivo Niskanen
20.        249   Kaapo Kakko
21.        222   Lewis Hamilton
22.        218   Matti Nykäsen
23.        213   Matti Nykäsen
24.        212   Krista Pärmäkoski
25.        203   Winnipeg Jets
26.        203   Sauli Niinistö
27.        201   Antti Rinteen
28.        189   Temptation Island
29.        189   Matti Latvala
30.        188   Valtteri Bottaksen
31.        188   Temptation Island

# [&laquo; Previous Lab](plotting_frequencies.ipynb)